# **1. Importing the Required Libraries**

In [ ]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# librosa is a Python library for analyzing audio and music.
# It can be used to extract the data from the audio files we will see it later
import librosa
import torch
import librosa.display
import warnings
warnings.filterwarnings("ignore")
# to play the audio files
from IPython.display import Audio
plt.style.use('seaborn-white')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2. Audio Feature Extraction using wavLM**

In [ ]:
import torchaudio


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoProcessor, WavLMModel, Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")
wavlm = WavLMModel.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")# audio file is decoded on the fly

def extract_features(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    input = processor(array.squeeze(), sampling_rate= sample_rate, return_tensors="pt")
    # apply the model to the input array from wav
    with torch.no_grad():
       outputs = wavlm(**input)
    # extract last hidden state, compute average, convert to numpy
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).numpy()
    return last_hidden_states

## **2.1 Training Dataset**

## **2.1.1 Dataset loading**

In [ ]:
csv_file_path = '/content/drive/MyDrive/Audio_Data/annotations/annotations/Odia_train.csv'
# Read the CSV file into a DataFrame
odia_train = pd.read_csv(csv_file_path)
odia_train.head()

,filename,label
0,Abuse_1.wav,Yes
1,Abuse_2.wav,No
2,Abuse_4.wav,Yes
3,Abuse_5.wav,Yes
4,Abuse_6.wav,Yes


In [ ]:
odia_train.shape

(814, 2)

## **2.1.2 Feature Extraction**

In [ ]:
image_directory_odia = '/content/drive/MyDrive/Audio_Data/Indian_Language_Audio_Dataset/SC_audio_Odia'
odia_train['filename']= image_directory_odia + "/" + odia_train['filename']
odia_train.head()

,filename,label
0,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
1,/content/drive/MyDrive/Audio_Data/Indian_Langu...,No
2,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
3,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
4,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes


In [ ]:
#save embedding in Numpy


In [ ]:
odia_train_embeddings = []
for i in range(len(odia_train)):
    features = extract_features(odia_train['filename'][i])
    odia_train_embeddings.append(features)

odia_train_embeddings = np.array(odia_train_embeddings)
print(odia_train_embeddings.shape)

(814, 768)


In [ ]:
np.save('/content/drive/MyDrive/Audio_Data/Output/odia_train_embeddings.npy',odia_train_embeddings)

print("odia_train_embeddings.npy file is saved")


odia_train_embeddings.npy file is saved


## **2.2 Testing Dataset**

## **2.2.1 Dataset Loading**

In [ ]:
csv_file_path_test = '/content/drive/MyDrive/Audio_Data/annotations/annotations/Odia_test.csv'
# Read the CSV file into a DataFrame
odia_test = pd.read_csv(csv_file_path_test)
odia_test.head()

,filename,label
0,Abuse_0.wav,No
1,Abuse_3.wav,No
2,Abuse_7.wav,Yes
3,Abuse_11.wav,Yes
4,Abuse_14.wav,Yes


In [ ]:
odia_test['filename']= image_directory_odia + "/" + odia_test['filename']
odia_test.head()

,filename,label
0,/content/drive/MyDrive/Audio_Data/Indian_Langu...,No
1,/content/drive/MyDrive/Audio_Data/Indian_Langu...,No
2,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
3,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
4,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes


In [ ]:
feats_test = np.array(extract_features(odia_test['filename'][0]))
feats_test.shape

(768,)

## **2.2.2 Feature Extraction**

In [ ]:
odia_test_embeddings = []
for i in range(len(odia_test)):
    features_test = extract_features(odia_test['filename'][i])
    odia_test_embeddings.append(features_test)

odia_test_embeddings = np.array(odia_test_embeddings)
print(odia_test_embeddings.shape)

(365, 768)


In [ ]:
np.save('/content/drive/MyDrive/Audio_Data/Output/odia_test_embeddings.npy',odia_test_embeddings)

print("odia_test_embeddings.npy file is saved")

odia_test_embeddings.npy file is saved
